In [2]:
import numpy as np
# import time
from standardbots import StandardBotsRobot, models

# ⬇⬇⬇⬇⬇⬇⬇⬇⬇⬇⬇⬇ Here ⬇⬇⬇⬇⬇⬇⬇⬇⬇⬇⬇⬇
# The URL is coorect now!!!!
# ⬆⬆⬆⬆⬆⬆⬆⬆⬆⬆⬆⬆      ⬆⬆⬆⬆⬆⬆⬆⬆⬆⬆⬆⬆
sdk = StandardBotsRobot(
    url="192.168.110.5:3000",
    token="oetrwf0e-yyquw-8eopsk-z8egwu6g",
    robot_kind=StandardBotsRobot.RobotKind.Live,
)

Q_IDENTITY = models.Quaternion(0.0, 0.0, 0.0, 1.0)

# Move tooltip to (x,y,z) in millimeters, orientation fixed.
def move_tooltip_xyz(x_mm: float, y_mm: float, z_mm: float):
    body = models.ArmPositionUpdateRequest(
        kind=models.ArmPositionUpdateRequestKindEnum.TooltipPosition,
        tooltip_position=models.PositionAndOrientation(
            position=models.Position(
                x=float(x_mm),
                y=float(y_mm),
                z=float(z_mm),
                unit_kind=models.LinearUnitKind.Millimeters,
            ),
            orientation=models.Orientation(
                kind=models.OrientationKindEnum.Quaternion,
                quaternion=Q_IDENTITY,
            ),
        ),
    )
    sdk.movement.position.set_arm_position(body=body).ok()

# Generate XY points for the curve
def generate_pattern_points(
    x0_mm: float, y0_mm: float, scale: float, n: int,
):
    # Original x, y, scaling value and sample points
    theta = np.linspace(0.0, 2.0 * np.pi, n, endpoint=True)
    # left start, right start, sample number, include last point(2 theta)
    # First point is always the first point
    r = np.sin(8.0 * theta) + np.cos(8.0 * theta)
    x = x0_mm + scale * r * np.cos(theta)
    y = y0_mm + scale * r * np.sin(theta)
    return x, y

def run_dotted_line_pattern_2d(
    # Placement(original plint and scale (mm)
    x0_mm: float = 550.0,
    y0_mm: float = -225.0,
    scale: float = 625.0,

    # 2D plane
    z_mm: float = 0.0,

    # Sampling
    n: int = 1200,

    # How many loop to draw
    loop: int = 1,
):

    x, y = generate_pattern_points(x0_mm, y0_mm, scale, n)
    for variable in range(int(loop)):
        for xi, yi in zip(x, y):
            move_tooltip_xyz(xi, yi, z_mm)
            # time.sleep(0.01)


with sdk.connection():
    run_dotted_line_pattern_2d(
        x0_mm = 550,
        y0_mm = -225,
        scale = 625,
        z_mm=0,

        n = 1200,
        loop = 1,
    )
    
# Two limitations casue this code not working:
# 1. The robotic arm is currently wobbling when it moves, 
# and the distance of each single displacement is too short (due to excessive sampling).
# 2. The robotic arm moved outside the range.


Exception: Request failed with status 503: ErrorResponse(error=<ErrorEnum.RobotNotIdle: 'robot_not_idle'>, message='Robot must be idle to perform this action (current state: Failure)')